## Overview
```
Author: Florian Wagner
Email: florian.wagner@duke.edu
```
This notebook pre-processes the DMAP expression data, which involves 1) mapping of Entrez Gene IDs to gene names and 2) filtering the genes against the set of protein-coding genes (derived from Ensembl genome annotations).

### Programs and thid-party Python packages used

In [1]:
from pkg_resources import require

print 'Python:'
!python -V
print

print 'Python packages:'
print str(require('genometools')[0])

Python:
Python 2.7.9

Python packages:
genometools 1.2rc5


## Read config file (config.ini)

In [2]:
import codecs
from configparser import ConfigParser

config_file = 'config.ini'

config = ConfigParser()
with codecs.open(config_file, 'rb', encoding = 'UTF-8') as fh:
    config.read_file(fh)
    
config = config['Demo']

In [3]:
# create output directory, if necessary
import os
if not os.path.isdir(config['output_dir']):
    os.mkdir(config['output_dir'])

## Extract Entrez ID -> gene name mapping
To generate a list of all human protein-coding genes, we're using the script `ncbi_extract_entrez2gene.py` from the [genometools](http://genometools.readthedocs.org/en/latest/) Python package.

In [4]:
!ncbi_extract_entrez2gene.py -f "{config['gene2accession_file']}" -o "{config['entrez2gene_file']}"

[2016-01-19 15:04:36] INFO: Found 57244 Entrez Gene IDs associated with 57231 gene symbols.
[2016-01-19 15:04:36] INFO: Output written to file "./output/entrez2gene_human.tsv".


## Extract list of human protein-coding genes
To generate a list of all human protein-coding genes, we're using the script `ensembl_extract_protein_coding_genes.py` from the [genometools](http://genometools.readthedocs.org/en/latest/) Python package.

In [5]:
!gunzip -c "{config['genome_annotation_file']}" | \
    ensembl_extract_protein_coding_genes.py -a - -o "{config['genome_file']}"

[2016-01-19 15:04:36] INFO: Regular expression used for filtering chromosome names: "(?:\d\d?|MT|X|Y)$"
[2016-01-19 15:04:36] INFO: Parsing data...
[2016-01-19 15:05:05] INFO: done (parsed 2569155 lines).
[2016-01-19 15:05:05] INFO: 
[2016-01-19 15:05:05] INFO: Gene chromosomes (25):
[2016-01-19 15:05:05] INFO: 	1, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 2, 20, 21, 22, 3, 4, 5, 6, 7, 8, 9, MT, X, Y
[2016-01-19 15:05:05] INFO: 
[2016-01-19 15:05:05] INFO: Excluded chromosomes (15):
[2016-01-19 15:05:05] INFO: 	GL000009.2, GL000194.1, GL000195.1, GL000205.2, GL000213.1, GL000218.1, GL000219.1, KI270711.1, KI270713.1, KI270721.1, KI270726.1, KI270727.1, KI270728.1, KI270731.1, KI270734.1
[2016-01-19 15:05:05] INFO: 
[2016-01-19 15:05:05] INFO: Gene sources:
[2016-01-19 15:05:05] INFO: 	ensembl_havana: 18825
[2016-01-19 15:05:05] INFO: 	havana: 791
[2016-01-19 15:05:05] INFO: 	ensembl: 226
[2016-01-19 15:05:05] INFO: 	insdc: 13
[2016-01-19 15:05:05] INFO: 
[2016-01-19 15:05:05] INFO: Gene 

## Filter genes in DMAP dataset
We're using the Entrez ID -> gene name mapping and the list of protein-coding genes generated above to filter the genes (rows) in the DMAP expression matrix.

In [6]:
import unicodecsv as csv

import numpy as np

from genometools.expression import ExpGenome, ExpMatrix
from genometools import misc

genome_file = config['genome_file']
entrez2gene_file = config['entrez2gene_file']
expression_file = config['expression_file']
mapped_expression_file = config['mapped_expression_file']

def read_dmap_expression(fn):
    entrez = []
    genes = []
    expr = []
    with open(fn) as fh:
        reader = csv.reader(fh, dialect='excel-tab')
        reader.next()
        p, n = [int(f) for f in reader.next()]
        samples = reader.next()[2:]
        assert len(samples) == n
        for l in reader:
            entrez.append(l[0])
            genes.append(l[1])
            expr.append(l[2:])
        assert len(genes) == p
    E = np.float64(expr)
    return entrez, genes, samples, E

genome = ExpGenome.read_tsv(genome_file)
e2g = dict(misc.read_all(entrez2gene_file))
entrez_dmap, genes_dmap, samples_dmap, X_dmap = read_dmap_expression(expression_file)

p, n = X_dmap.shape
unknown_entrez = 0
genes = []
indices = []
for i in range(p):
    try:
        gene = e2g[entrez_dmap[i]]
    except KeyError:
        unknown_entrez += 1
        pass
    else:
        if gene in genome:
            genes.append(gene)
            indices.append(i)

indices = np.int64(indices)
X = X_dmap[indices,:]

print 'Number of unknown Entrez IDs: %d / %d (%.1f%%)' \
        %(unknown_entrez, p, 100*(unknown_entrez / float(p)))

print 'Number of successfully mapped genes: %d / %d (%.1f%%)' \
        %(len(genes), p, 100*(len(genes) / float(p)))

E = ExpMatrix(genes, samples_dmap, X)
E.write_tsv(mapped_expression_file)

Number of unknown Entrez IDs: 147 / 8968 (1.6%)
Number of successfully mapped genes: 8547 / 8968 (95.3%)


## Copyright and License

Copyright (c) 2016 Florian Wagner.

This work is licensed under a [Creative Commons Attribution-NonCommercial-ShareAlike 4.0 International License](http://creativecommons.org/licenses/by-nc-sa/4.0/).